In [1]:
# Some part of the code is taken from https://www.kaggle.com/code/sharifashik/predict-bangla-newspaper-category-by-lstm

import numpy as np
import pandas as pd

from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder

from bnlp import BasicTokenizer
from bnlp.corpus import stopwords, punctuations, letters, digits

import json
import pickle

/home/fahim/anaconda3/envs/explainable-news/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/fahim/anaconda3/envs/explainable-news/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
with open(file='data/data_v2.json', encoding='utf-8') as file:
    data=json.load(file)

In [3]:
all_category=[]

for i in range(len(data)):
    
    cat=data[i]['category']
    
    all_category.append(cat)

unique_cat=np.unique(np.array(all_category))

print(len(unique_cat))

9


In [4]:
category_count=[]
for category in unique_cat:
    x=all_category.count(category)
    category_count.append((category,x))

category_count.sort()
category_count

[('bangladesh', 232504),
 ('economy', 17245),
 ('education', 9721),
 ('entertainment', 30466),
 ('international', 30856),
 ('life-style', 10852),
 ('opinion', 15699),
 ('sports', 49012),
 ('technology', 12116)]

In [6]:
feature=[]
target=[]
headline = []

for p in data:
    if p['category'] in unique_cat:
        headline.append(p['title'])
        feature.append(p['content'])
        target.append(p['category'])

In [7]:
data_frame = pd.DataFrame()

data_frame['feature'] = feature
data_frame['target'] = target
data_frame['title'] = headline

In [8]:
data_frame.head()

,feature,target,title
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,bangladesh,"কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ, বিক..."
1,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...,sports,সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি
2,জাতীয় সংসদে বিএনপি চেয়ারপারসন ও বিরোধীদলীয় ...,bangladesh,সংসদে খালেদার অভিযোগের জবাব দিয়েছে ভারত
3,সহজ পাসওয়ার্ডের কারণে অনলাইন অ্যাকাউন্ট সহজেই...,technology,পাসওয়ার্ড ভুলে যান!
4,কম্পিউটার মাউসের উদ্ভাবক ডগলাস অ্যাঙ্গেলবার্ট ...,technology,চলে গেলেন মাউস উদ্ভাবক


In [9]:
label_enc = LabelEncoder()

In [10]:
data_frame['label'] = label_enc.fit_transform(data_frame['target'])

In [11]:
data_frame.head()

,feature,target,title,label
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,bangladesh,"কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ, বিক...",0
1,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...,sports,সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি,7
2,জাতীয় সংসদে বিএনপি চেয়ারপারসন ও বিরোধীদলীয় ...,bangladesh,সংসদে খালেদার অভিযোগের জবাব দিয়েছে ভারত,0
3,সহজ পাসওয়ার্ডের কারণে অনলাইন অ্যাকাউন্ট সহজেই...,technology,পাসওয়ার্ড ভুলে যান!,8
4,কম্পিউটার মাউসের উদ্ভাবক ডগলাস অ্যাঙ্গেলবার্ট ...,technology,চলে গেলেন মাউস উদ্ভাবক,8


In [12]:
label_enc.classes_

array(['bangladesh', 'economy', 'education', 'entertainment',
       'international', 'life-style', 'opinion', 'sports', 'technology'],
      dtype=object)

In [13]:
bangla_tokenizer = BasicTokenizer()

In [14]:
def clean_text(samples):
    
    tokens = bangla_tokenizer.tokenize(samples)
    
    temp = []
    
    
    for i in tokens:
        
        if i in stopwords:
            
            continue
            
        if i in punctuations + '‘' + '’':
            
            continue
            
        temp.append(i)
        
    return " ".join(temp)

In [15]:
data_frame['feature'] = data_frame['feature'].apply(clean_text)

In [16]:
data_frame['title'] = data_frame['title'].apply(clean_text)

In [18]:
data_frame.head()

,feature,target,title,label
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,bangladesh,কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ বিক্ষোভ,0
1,এবারের উইম্বলডনটা স্মরণীয় রাখার মিশনেই নেমেছে...,sports,সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি,7
2,জাতীয় সংসদে বিএনপি চেয়ারপারসন বিরোধীদলীয় নে...,bangladesh,সংসদে খালেদার অভিযোগের জবাব ভারত,0
3,সহজ পাসওয়ার্ডের কারণে অনলাইন অ্যাকাউন্ট সহজেই...,technology,পাসওয়ার্ড ভুলে,8
4,কম্পিউটার মাউসের উদ্ভাবক ডগলাস অ্যাঙ্গেলবার্ট ...,technology,গেলেন মাউস উদ্ভাবক,8


In [ ]:
data.to_csv(f'data/cleaned_data.csv', index = False)

In [21]:
with open(f'label_encoder.pkl', 'wb') as f:
    
    pickle.dump(label_enc, f)
    
    f.close()